In [ ]:
import os
import sys
import pickle
import json
import numpy as np
import torch
import matplotlib as mpl
import matplotlib.pyplot as plt
sys.path.append(os.path.dirname(os.getcwd()))


device = torch.device("cpu")

In [ ]:
from src.dynamics import KolmogorovFlow

grid_size = 128
reynolds = 1e3
dt = 0.2
seed = 42

dynamics = KolmogorovFlow(
    grid_size=grid_size,
    reynolds=reynolds,
    dt=dt,
    seed=seed,
)

folders = [
    "../kolmogorov_results/kolmogorov-241102-050922",
    "../kolmogorov_results/kolmogorov-241102-050952",
    "../kolmogorov_results/kolmogorov-241102-045641",
]

df = []
for workdir in folders:
    with open(os.path.join(workdir, 'config.json'), 'r') as f:
        cfg = json.load(f)
    noise_std = cfg['measurement']['noise_std']
    kernel_size = cfg['measurement']['kernel_size']
    n_train = cfg['train']['n_train']
    results = np.load(os.path.join(workdir, 'results.npz'))
    states = results['states'] # (steps, 2, grid_size, grid_size)
    assimilated_states = results['assimilated_states'] # (steps, nsamples, 2, grid_size, grid_size)
    mean_estimation = np.mean(assimilated_states, axis=1) # (steps, 2, grid_size, grid_size)
    average_rmse = np.mean((states - mean_estimation)**2, axis=(1, 2, 3))**0.5 # (steps, )
    df.append({
        "kernel_size": kernel_size,
        "average_rmse": average_rmse,
    })

true_init = dynamics.prior(n_sample=1).cpu().numpy() # (1, 2, grid_size, grid_size)
guess_init = dynamics.prior(n_sample=cfg["train"]["n_train"]).cpu().numpy() # (n_train, 2, grid_size, grid_size)
mean_init = np.mean(guess_init, axis=0, keepdims=True) # # (1, 2, grid_size, grid_size)
init_rmse = np.mean((mean_init - true_init)**2, axis=(1, 2, 3))**0.5
with open("../asset/Kolmogorov_metrics.pkl", "wb") as file:
    pickle.dump((df, init_rmse), file)

In [ ]:
with open("../asset/Kolmogorov_metrics.pkl", "rb") as file:
    df, init_rmse = pickle.load(file)
mpl.rcdefaults()
mpl.style.use("../configs/mplrc")
mpl.rc("lines", marker='o', markersize=4, markerfacecolor='none')

fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(7, 3))
t = np.arange(52) * 0.2
for data in df:
    ax.plot(t, np.hstack((init_rmse, data["average_rmse"])), label=data["kernel_size"])
ax.set_xticks([0, 2, 4, 6, 8, 10])
ax.set_xlim([0, 11])
ax.grid(linestyle=':', alpha=0.8)
ax.legend(title="AvgPool scale")
ax.set_xlabel("Time step")
ax.set_ylabel("RMSE")
ax.set_yscale('log')
plt.savefig('../asset/Kolmogorov_metrics.pdf', dpi=600, bbox_inches='tight', pad_inches=0.1)